# Evalaution d'une mesure de gestion de trafic sur le barreau A20/A89

## Contexte de l’étude
L’autoroute A20, analysée dans le cadre de ce TP, s’étend de Vierzon à Montauban, et supporte le trafic de transit Paris – Toulouse. Dans le secteur de Brive-la-Gaillarde, le trafic de l’autoroute A89 emprunte pendant une vingtaine de kilomètres celui l’autoroute A20. Ce barreau A20/A89 longe la ville de Brive-la-Gaillarde. Il est constitué d’une section à deux fois deux voies,  avec voie lente limitée à 110 km/h du PR 261+900 (point A) au PR 266+000 (point B)


![title](1.png)

La section présente des trafics modérés, en semaine sur les trafics pendulaires comme en week-end. En revanche, lors des vacances estivales convergent les transits nord-sud (Paris – Toulouse) Le barreau autoroutier supporte donc une forte demande de trafic sur cette période et présente une congestion recensée de 10 h à 17 h sur les samedis de juillet
À cela s’ajoutent des contraintes de relief importantes avec le franchissement du plateau du Limousin. Par ailleurs, la voie de contournement nord de l’agglomération débouche au niveau de l’échangeur 49, à l’intérieur du barreau commun.

La Direction Interdépartementale des Routes Centre-Ouest (DIRCO), exploitant de l’A20, a mis en place une mesure d’exploitation spécifique visant à fluidifier le trafic sur l’autoroute A20, dans sa partie commune avec l’autoroute A89, durant deux samedis de juillet 2019. L’objectif étant d’améliorer le service à l’usager en réduisant le volume des congestions sur le secteur durant cette période. Ces stratégies de régulation de trafic ont été appliquées à partir du vendredi matin jusqu’au lundi matin sur le deuxième et troisième week-end de juillet 2019.

Ces stratégies sont les suivantes :
- Abaissement de la vitesse de 110 km/h à 90 km/h entre le PR 261+500 et le PR 269+850 ;
- Interdiction de dépassement des véhicules poids lourds entre le PR 261+500 et le PR 266+000 ;
- Circulation interdite sur la voie de gauche du PR 261+900 au PR 266+000 ;
- Circulation rabattue sur la voie de droite et la voie spéciale pour véhicules lents du PR 261+900 au PR 266+000 ;
- Circulation sur la voie spéciale pour véhicules lents a été autorisée du PR 261+900 au PR 266+000 à toutes les catégories de véhicules à l’exception de ceux dont l’accès des autoroutes est interdit ;
-  Affichage d’un message de restriction de voie par la gauche ( avec pictogramme de passage de 3 à 2voies ) sur e PMV en amont au PR 256 ;
- Mise en place d’un panneau de signalisation fixe indiquant aux usagers l’autorisation de circulation sur la voie réservée aux véhicules lents.

L’ensemble de ces mesures est reporté sur la carte suivante.
CEtte carte présente également le dispositif de recueil de données qui a été mis en place pour l’évaluation a posteriori de ces mesures. On notera l’utilisation des quatre boucles électromagnétiques nommées SRDT46_47, SRDT47_48, SRDT48_49 et SRDT49 ainsi que des 31 tronçons FCD de 500 m numérotés de 1 à 31 dans le sens Nord-Sud.

![title](2.jpg)

Afin d’évaluer la pertinence de la mesure, vous êtes chargé d'effectuer une mission d’évaluation. Cette mission est une étude à posteriori de la mesure de gestion de trafic mise en place par la DIRCO. Elle devra se concentrer sur la section de l’A20 présentant les plus fortes perturbations en période estivale sur les samedis des mois de juillet 2018 et 2019.
N'oubliez pas que l’objectif principal de la mesure consiste à fluidifier le barreau autoroutier durant la période estivale et en particulier les samedis de juillet.


In [9]:
# Un peu d'aide au démarrage : voici l'import de quelques modules dont vous aurez besoin pour réaliser votre étude ...
import sys
!{sys.executable} -m pip install pandas matplotlib
import pandas as pd
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
from datetime import timedelta
import matplotlib.ticker as ticker
import matplotlib.style as style
import numpy as np

In [10]:
#%% ... et voici l'import données de trafic
df=pd.read_csv('dfAll.csv', sep=';', index_col = 0, dtype = {'id capteur':str,'voie':str,'sens':str,'date':str,'agregation':int,'type vehicule':str,'id vehicule':str,'debit':int,'vitesse':int,'nb essieux':int,'longueur vehicule':float,'taux occupation':float,'temps presence':float,'distance intervehiculaire':float})

# On supprime MB133Q (capteur d'un autre étude)
df=df[df["id capteur"]!='MB133.q']

# On met tout en datetime et on créer une colonne jour et heure
df['date']=pd.to_datetime(df['date'],format='%Y/%m/%d %H:%M:%S')
df['heure']=df['date'].apply(lambda x: x.strftime('%H:%M'))
df['jour']=df['date'].apply(lambda x: x.strftime('%Y/%m/%d'))

# On définit les jours avec et sans mesure
jour_sans_mesure_2019=pd.to_datetime(('2019-07-06','2019-07-27'),format='%Y/%m/%d')
jour_avec_mesure_2019=pd.to_datetime(('2019-07-13','2019-07-20'),format='%Y/%m/%d')
liste_str_jour_avec_mesure_2019=[item.strftime("%Y/%m/%d") for item in jour_avec_mesure_2019]
jour_sans_mesure_2018=pd.to_datetime(('2018-07-07','2018-07-14','2018-07-21','2018-07-28'),format='%Y/%m/%d')
liste_jour_total=list(jour_sans_mesure_2019)+list(jour_avec_mesure_2019)+list(jour_sans_mesure_2018)
liste_str_jour_total=[item.strftime("%Y/%m/%d") for item in liste_jour_total]


# On ajout certaines colonnes
df['type_donnees']=df['id capteur'].apply(lambda x: "FCD" if "A20_sens" in x else "Boucles" ) # Type des données
df['utile']=df['jour'].apply(lambda x: True if x in liste_str_jour_total else False) # jour qui nous interesse dans l'étude
df['2019']=df['date'].apply(lambda x: True if x.strftime("%Y")=="2019" else False)
df=df[df.utile==True]# On supprime les jours inutiles
df['mesure_active']=df['jour'].apply(lambda x: True if x in liste_str_jour_avec_mesure_2019 else False)

#df.to_csv('data_DIRCO_A20.csv')

# On liste les capteurs et les dates différentes
liste_capteur=list(df["id capteur"].unique())
liste_jour=list(df["jour"].unique())










## Petite note concernant le calcul des temps de parcours de la section

Les données de temps de parcours des usagers ont été calculé à partir des données FCD. Pour ce faire, nous avons calculé un temps de traversée de la section complète du barreau A20/A89 dans le sens Nord-Sud, depuis le tronçon FCD n°1 jusqu’au tronçon n°31.

Méthodologie de calcul :
Pour ce calcul, nous avons simulé une entrée d’un usager sur la section du réseau routier toutes les minutes puis nous avons effectué les actions suivantes :
    • recherche de la valeur de la vitesse moyenne associée au premier tronçon FCD du réseau à la date d’entrée de l’usager ;
    • recherche de la date de sortie de l’usager pour ce premier tronçon à partir de sa valeur de  vitesse et de sa longueur (500 m). Ainsi nous pouvons en déduire sa date de sortie du premier tronçon ;
    • répétition de la recherche précédente sur les tronçons suivants en prenant en compte comme date d’entrée du tronçon suivant la date de sortie du tronçon précédent ;
    • calcul du temps de parcours de cet usager sur l’ensemble des tronçons FCD : le temps de parcours global est égal à la somme des temps de parcours de chaque tronçon ;
    • agrégation temporelle sur une période de 6 minutes de la valeur de temps de parcours d’un usager entrant sur la section du réseau routier calculé toutes les minutes.
    
La figure ci-dessous permet de représenter la méthodologie de calcul d'une moyenne des temps de parcours pour une minute. Cette valeur moyenne de temps de parcours, agrégée toutes les 6 minutes, sera prise comme valeur de temps de parcours sur la section (voir variable data_TP)

![title](3.jpg)

In [11]:
data_TP=pd.read_csv('dataTP_aggr.csv')
data_TP

,heure_aggr,TP_s,date,heure,jour
0,2018-07-07 00:00:00,490.228063,2018-07-07 00:00:00,00:00,2018/07/07
1,2018-07-07 00:06:00,483.968833,2018-07-07 00:06:00,00:06,2018/07/07
2,2018-07-07 00:12:00,473.258870,2018-07-07 00:12:00,00:12,2018/07/07
3,2018-07-07 00:18:00,449.254511,2018-07-07 00:18:00,00:18,2018/07/07
4,2018-07-07 00:24:00,482.761022,2018-07-07 00:24:00,00:24,2018/07/07
...,...,...,...,...,...
1915,2019-07-27 23:30:00,510.558982,2019-07-27 23:30:00,23:30,2019/07/27
1916,2019-07-27 23:36:00,523.908755,2019-07-27 23:36:00,23:36,2019/07/27
1917,2019-07-27 23:42:00,520.302233,2019-07-27 23:42:00,23:42,2019/07/27
1918,2019-07-27 23:48:00,540.393115,2019-07-27 23:48:00,23:48,2019/07/27


In [ ]:
df

### Maintenant c'est à vous de jouer !

######   Théorie
1/ Etudier de manière théorique la section avec une demande constante de 2500 veh/h pendant une heure puis une demande constante de 3500 veh/h pendant une heure. On prendra les valeurs par défauts présentées en slide 16 du document 2_Théorie_traffic.pptx

###### Pratique - données SIREDO SRDT seules
2/ Etudier les informations de contexte permettent d’affiner l’analyse statistique des données (par exemple des informations relatives à la météo) et vérifier s'il ne faut pas supprimer un jour de données qui présente des caractéristique très spécifique.
3/ Vérifier si les données réelles présentent ou non des erreurs de mesures et expliciter les tests effectués (par exemple si vitesse supérieure à 150 km/h => données à suprimer)
4/ A partir des données réelles, étudier les caractéristiques de l'infrastructures (vitesse maximale, moyenne, débit moyen journalier, proportion de PL ...) en comparant 2018 et 2019
5/ Afficher les diagrammes fondamentaux pour chaque boucle SIREDO pour 2018 et 2019. Commenter

###### Pratique - données FCD + SIREDO 
6/ Avant d’être utilisées dans l’étude, les données FCD doivent être qualifiées. Vérfier le taux de pénétration des FCD (nombre de véhicules émmetant des données FCD avec leur GPS diviser par le nombre de véhicules total du trafic mesuré par les boucles SIREDO) dans le trafic, comparativement aux données de boucles prises comme référence. On considère qu'une représentativité de 3% est suffisante. On prendra pour chaque boucle SIREDO le tronçon FCD correspondant (par exemple SRDT_A20_46_47=> tronçon FCD 6 - voir la carte slide 3 du fichier TP_A20.pptx)
7/ Comparer les vitesses issues des boucles SIREDO et celles des données FCD
8/ Visualiser les données de vitesse FCD le long de l'infrastructure
9/ Analyser les temps de parcours des usagers de manière quantitative et avec des courbes (moyenne, écart type, minimum, maximum, distribution ...). Comparer les temps de parcours entre 2018 et 2019
10/ Conclure sur l'utilité de la mesure mise en place par la DIRCO

